In [ ]:
import gc, os, sys, traceback
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
def write_error(e):
    with open('error.log', 'a') as f:
        f.write(f'\nERROR: {str(e)}\n')
        traceback.print_exc(file=f)
    print(f'Error: {e}')
sys.excepthook = lambda t, v, tb: write_error(v)

In [ ]:
import torch
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install -q qwen-tts==0.1.1 soundfile

In [ ]:
TEXT = "Success is the goal."
VOICE_ID = "Ryan"
OUTPUT_FILE = "voicegenhub_output.wav"

In [ ]:
from qwen_tts import Qwen3TTSModel
print('Loading Qwen TTS model...')
model = Qwen3TTSModel.from_pretrained(
    'Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice',
    torch_dtype=torch.float16,
    device_map='auto'
)

In [ ]:
print(f'Generating with do_sample=False and subtalker_dosample=False for stability...')
try:
    res = model.generate_custom_voice(
        text=TEXT,
        speaker=VOICE_ID,
        do_sample=False,
        subtalker_dosample=False
    )
except Exception as e:
    print(f'Generation failed for {VOICE_ID}: {e}. Trying fallback.')
    res = model.generate_custom_voice(
        text=TEXT,
        speaker='ryan',
        do_sample=False,
        subtalker_dosample=False
    )
print(f'Raw res type: {type(res)}')


In [ ]:
print(f'Processing output...')
import numpy as np

def to_numpy(x):
    if isinstance(x, torch.Tensor): return x.detach().cpu().numpy()
    if isinstance(x, (list, tuple)): return np.array(x)
    return x

if isinstance(res, (list, tuple)):
    audio_data = to_numpy(res[0])
    sr = res[1] if len(res) > 1 and isinstance(res[1], (int, float)) else 24000
else:
    audio_data = to_numpy(res)
    sr = 24000

audio_data = to_numpy(audio_data)
if hasattr(audio_data, 'ndim') and audio_data.ndim > 1: audio_data = audio_data.squeeze()
print(f'Final audio shape: {getattr(audio_data, "shape", len(audio_data))}, SR: {sr}')

In [ ]:
import soundfile as sf
import numpy as np
data = audio_data
if isinstance(data, torch.Tensor):
    data = data.detach().cpu().numpy()
if data.ndim > 1:
    data = data.squeeze()
sf.write(OUTPUT_FILE, data, int(sr))
print(f'Saved to {OUTPUT_FILE}')

In [ ]:
del model
torch.cuda.empty_cache()
print('Cleaned up GPU')